## DIA 095: Exportar Historial Filtrado por Rango de Fechas a CSV (Flask + Pandas)

Después de aprender a filtrar predicciones por fecha, hoy implementarás una función que permite al usuario descargar un archivo CSV solo con los registros dentro del rango de fechas seleccionado. Esto mejora la utilidad del historial y permite al usuario analizar sus datos en Excel, R o cualquier herramienta externa.

✅ ¿Qué aprenderás hoy?

📌 Cómo recibir fechas desde un formulario.

📌 Cómo filtrar los datos usando SQLAlchemy.

📌 Cómo generar y enviar un archivo .csv personalizado.

🧩 1. Ruta en app.py: /exportar-csv-filtrado
Agrega esto en tu archivo principal:

python
Copiar
Editar
from flask import request, send_file
import pandas as pd
import io

@app.route('/exportar-csv-filtrado', methods=['GET', 'POST'])
@login_required
def exportar_csv_filtrado():
    if request.method == 'POST':
        fecha_inicio = request.form['fecha_inicio']
        fecha_fin = request.form['fecha_fin']

        if not fecha_inicio or not fecha_fin:
            return "⚠️ Ambas fechas son obligatorias", 400

        try:
            f1 = datetime.strptime(fecha_inicio, '%Y-%m-%d')
            f2 = datetime.strptime(fecha_fin, '%Y-%m-%d')

            registros = Registro.query.filter(
                Registro.user_id == current_user.id,
                Registro.fecha >= f1,
                Registro.fecha <= f2
            ).order_by(Registro.fecha.desc()).all()

            if not registros:
                return "⚠️ No se encontraron registros en ese rango", 404

            data = [{
                "Fecha": r.fecha.strftime('%Y-%m-%d %H:%M:%S'),
                "F1": r.f1,
                "F2": r.f2,
                "F3": r.f3,
                "F4": r.f4,
                "Predicción": r.prediccion
            } for r in registros]

            df = pd.DataFrame(data)
            output = io.StringIO()
            df.to_csv(output, index=False)
            output.seek(0)

            return send_file(
                io.BytesIO(output.getvalue().encode()),
                mimetype='text/csv',
                as_attachment=True,
                download_name=f"historial_{fecha_inicio}_a_{fecha_fin}.csv"
            )

        except Exception as e:
            return f"❌ Error procesando fechas: {str(e)}", 500

    return render_template('exportar_csv_filtrado.html')
🖼️ 2. Template: exportar_csv_filtrado.html
html
Copiar
Editar
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <title>Exportar CSV Filtrado</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body class="bg-light">
<div class="container mt-5">
    <h2 class="text-center mb-4">📥 Exportar Historial Filtrado a CSV</h2>
    <form method="POST" class="row g-3">
        <div class="col-md-5">
            <label class="form-label">Desde:</label>
            <input type="date" class="form-control" name="fecha_inicio" required>
        </div>
        <div class="col-md-5">
            <label class="form-label">Hasta:</label>
            <input type="date" class="form-control" name="fecha_fin" required>
        </div>
        <div class="col-md-2 align-self-end">
            <button type="submit" class="btn btn-success w-100">Exportar</button>
        </div>
    </form>
    <a href="/mi-historial" class="btn btn-outline-secondary mt-4">← Volver</a>
</div>
</body>
</html>
🔗 3. Agrega botón de acceso en otras vistas
Por ejemplo, en mi_historial.html puedes poner:

html
Copiar
Editar
<a href="/exportar-csv-filtrado" class="btn btn-outline-success">📤 Exportar por fecha</a>
🔍 Líneas Clave Explicadas
Registro.query.filter(...): filtra predicciones por usuario y rango de fechas.

df.to_csv(...): crea el archivo CSV desde un DataFrame.

send_file(...): lo envía como archivo descargable.

request.form['fecha_inicio']: recoge las fechas desde el formulario.